This is a notebook to look at two-table verbs to combine tables and functions to reshape tables from "wide" to "long" and vice versa.

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Two-table-verbs-for-combining-data" data-toc-modified-id="Two-table-verbs-for-combining-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Two-table verbs for combining data</a></div><div class="lev2 toc-item"><a href="#Inner-joins" data-toc-modified-id="Inner-joins-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Inner joins</a></div><div class="lev2 toc-item"><a href="#Left-joins" data-toc-modified-id="Left-joins-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Left joins</a></div><div class="lev2 toc-item"><a href="#Right-joins" data-toc-modified-id="Right-joins-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Right joins</a></div><div class="lev2 toc-item"><a href="#Full-joins" data-toc-modified-id="Full-joins-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Full joins</a></div><div class="lev2 toc-item"><a href="#Anti-joins" data-toc-modified-id="Anti-joins-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Anti joins</a></div><div class="lev1 toc-item"><a href="#Reshaping-data" data-toc-modified-id="Reshaping-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reshaping data</a></div><div class="lev2 toc-item"><a href="#Spreading-data-(from-long-to-wide)" data-toc-modified-id="Spreading-data-(from-long-to-wide)-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Spreading data (from long to wide)</a></div><div class="lev2 toc-item"><a href="#Gathering-data-(from-wide-to-long)" data-toc-modified-id="Gathering-data-(from-wide-to-long)-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Gathering data (from wide to long)</a></div>

# Two-table verbs for combining data

In [1]:
library(tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages -----------------------------------------------------------------------------------------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


Let's start by creating two tables with a common column name, `x1`.

In [2]:
df1 <- tibble(x1=c('A','B','C','E','A'),
              x2=c(1,2,3,4,5))
df2 <- tibble(x1=c('A','B','D'),
              x3=c(T,F,T))
df1
df2

x1,x2
A,1
B,2
C,3
E,4
A,5


x1,x3
A,TRUE
B,FALSE
D,TRUE


## Inner joins

Inner joins return all matches between two tables.

For instance, the below keep rows that have matches in both `df1` and `df2` on column `x1`.
(The trailing select and arrange are just to order things for comparison between operations.)

In [3]:
inner_join(df1, df2) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE


Inner joins drop any rows that don't have common keys between the two tables. For instance, there are `E` and `C` rows in `df1` and a `D` row in `df2`, but these all get dropped. Only the `A` and `B` rows stick around.

So if you're not careful when you inner join, sometimes you'll drop rows you don't mean to.

Also, notice the warning: `inner_join` implicitly joined on *all* columns that `df1` and `df2` have in common. We could have been a bit more careful and specified this ourselves instead using the `by` argument.

In [4]:
inner_join(df1, df2, by=c('x1'='x1')) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE


Inner joins are symmetric, meaning that it doesn't matter what order you specify the arguments in.

In [5]:
# inner join is symmetric
inner_join(df2, df1) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE


Inner joins guarantee that we get all keys (here an `x1` value) that lie at the intersection of the keys for the two individual tables.

In [6]:
all(unique(inner_join(df1, df2)$x1) == intersect(df1$x1, df2$x1))

Joining, by = "x1"


[1] TRUE

## Left joins

Left joins are similar, but ensure that you don't drop any rows from the left table, even if they don't have a match in the right table. Use them when you want to augment an important table (the left one) with some auxiliary information (in the right one).

Unmatched rows will have missing entries in the result.

In [7]:
left_join(df1, df2) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE
C,3,NA
E,4,NA


Even though the rows with `C` and `E` in `df1` don't have a match, they're still returned in the results.

Left joins aren't symmetric.
For instance, here we'll keep all of the rows from `df2` instead.

In [8]:
left_join(df2, df1) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE
D,NA,TRUE


Inner joins guarantee that we get all keys (here an `x1` value) from the left table in the result.

In [9]:
all(unique(left_join(df1, df2)$x1) == unique(df1$x1))

Joining, by = "x1"


[1] TRUE

## Right joins

Right joins are just like left joins, but the table on the right is the one whose keys are preserved.

In [10]:
right_join(df1, df2) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE
D,NA,TRUE


This is identical to `left_join(df2, df1)`, shown above.

So really you only need one of these in your toolbox. To avoid confusion, just stick with left joins and switch the order when needed.

## Full joins

Full joins combine both safeguards of the left and right join, making sure no keys are dropped.

Unmatched rows have missing values in the result.

In [11]:
# match on values in column x1, keep all rows in both df1 and df2, and create NAs for unmatched entries
full_join(df1, df2) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE
C,3,NA
D,NA,TRUE
E,4,NA


And full joins are symmetric.

In [12]:
full_join(df2, df1) %>%
    select(x1, x2, x3) %>%
    arrange(x1, x2)

Joining, by = "x1"


x1,x2,x3
A,1,TRUE
A,5,TRUE
B,2,FALSE
C,3,NA
D,NA,TRUE
E,4,NA


Full joins guarantee that we get the union of all keys from both tables.

In [13]:
all(unique(full_join(df1, df2)$x1) == union(df1$x1, df2$x1))

Joining, by = "x1"


[1] TRUE

## Anti joins

Finally, we have the anti join, which shows the rows in the left table that don't have a match in the right one.

In [14]:
anti_join(df1, df2)

Joining, by = "x1"


x1,x2
E,4
C,3


The anti join isn't symmetric, returning a different result if we switch the arguments.

In [15]:
anti_join(df2, df1)

Joining, by = "x1"


x1,x3
D,TRUE


Why would you use an anti join?

Most commonly you'd use it to check that rows *weren't* dropped from the left table.

For instance, you could check that the number of rows in an anti join is zero.

In [16]:
nrow(anti_join(df1, df2)) == 0

Joining, by = "x1"


[1] FALSE

# Reshaping data

We've discussed the idea of tidy data: that each row is an observation, each column a variable, and each cell a value. Unfortunately we aren't always handed tidy data and sometimes have to clean things up ourselves.

This is where the `tidyr` package comes in handy.

For instance, let's say you were given the following table of state statistics.

In [17]:
states <- tibble(name = rep(state.name[1:3], times = 2),
                 dimension = rep(c('Area','Population'), each = 3),
                 stat = c(state.x77[1:3, "Area"], state.x77[1:3, "Population"] * 1e3))
states <- slice(states, 1:5)
states

name,dimension,stat
Alabama,Area,50708
Alaska,Area,566432
Arizona,Area,113417
Alabama,Population,3615000
Alaska,Population,365000


Notice that we've intentionally left out the row for Arizona's population.
We'll see why below.

## Spreading data (from long to wide)

This doesn't pass the tidy test because the `stat` column contains a concatenation of two differen variables. But we can easily change that with the `spread` function to take it from a "long" table to a "wide" one.

All we need to do is specify the column to use as the key (`dimension`) and the value (`stat`).
As a result, we get back one column per key, and values fill in the cells.

In [18]:
states_wide <- spread(states, dimension, stat)
states_wide

name,Area,Population
Alabama,50708,3615000
Alaska,566432,365000
Arizona,113417,NA


Notice the missing value in the last row, because the population of Arizona was missing from the original table.

## Gathering data (from wide to long)

Sometimes we have the opposite scenario, where columns contain the values of a variable.

For instance, let's say we had population by year, one year per column.

In [19]:
state_pop_by_year <- states_wide %>%
    mutate(year_1977 = Population,
           year_1978 = 1.05 * year_1977,
           year_1979 = 1.05 * year_1978) %>%
    select(name, year_1977, year_1978, year_1979)
names(state_pop_by_year) <- gsub('year_', '', names(state_pop_by_year))
state_pop_by_year

name,1977,1978,1979
Alabama,3615000,3795750,3985537.5
Alaska,365000,383250,402412.5
Arizona,NA,NA,NA


This doesn't pass the tidy test because variable values (here the year) are used as column names. But we can easily change that with the `gather` function to take it from a "wide" to a "long" table.

All we need to do is specify what we want to call the resulting key and value columns, along with which columns to collect.

In [20]:
gather(state_pop_by_year, "year", "population", 2:4)

name,year,population
Alabama,1977,3615000.0
Alaska,1977,365000.0
Arizona,1977,NA
Alabama,1978,3795750.0
Alaska,1978,383250.0
Arizona,1978,NA
Alabama,1979,3985537.5
Alaska,1979,402412.5
Arizona,1979,NA


Notice that the key and value arguments are *quoted strings*, not bare column names.
This is one of the only times you'll see this in the `tidyverse`, so watch out for it.

So this works, but there are a few awkward things about it.

First, we had to specify column numbers, which is never great---if we happen to change the order of things and want to re-run our code.
We could use the column names, but there are nice helper functions for this that let us choose all columns that match a pattern, for instance starting with `197`.

In [21]:
gather(state_pop_by_year, "year", "population", matches('^197?'))

name,year,population
Alabama,1977,3615000.0
Alaska,1977,365000.0
Arizona,1977,NA
Alabama,1978,3795750.0
Alaska,1978,383250.0
Arizona,1978,NA
Alabama,1979,3985537.5
Alaska,1979,402412.5
Arizona,1979,NA


Sometimes you'll see columns specified by a negation instead.
This works just as well, but can be confusing, so avoid if it possible.

In [22]:
gather(state_pop_by_year, "year", "population", -name)

name,year,population
Alabama,1977,3615000.0
Alaska,1977,365000.0
Arizona,1977,NA
Alabama,1978,3795750.0
Alaska,1978,383250.0
Arizona,1978,NA
Alabama,1979,3985537.5
Alaska,1979,402412.5
Arizona,1979,NA


Even with the `matches` operator, though, it can be difficult to parse what's going on here.

So here's a readability suggestion from the R4DS book: specify the arugments in a different order: first the table, then the columns to gather, and finally what you'd like to call the resulting two columns.

In [23]:
gather(state_pop_by_year, matches('^197?'), key = "year", value = "population")

name,year,population
Alabama,1977,3615000.0
Alaska,1977,365000.0
Arizona,1977,NA
Alabama,1978,3795750.0
Alaska,1978,383250.0
Arizona,1978,NA
Alabama,1979,3985537.5
Alaska,1979,402412.5
Arizona,1979,NA


You have a much better chance of remembering what that does when you read it a month or year from now.

Spread and gather are *almost* inverses of each other, the only side effect is that you'll end up with missing values for combinations that didn't exist in the original table.

In [24]:
states
gather(spread(states, dimension, stat), Area, Population, key = "dim", value = "statistic")

name,dimension,stat
Alabama,Area,50708
Alaska,Area,566432
Arizona,Area,113417
Alabama,Population,3615000
Alaska,Population,365000


name,dim,statistic
Alabama,Area,50708
Alaska,Area,566432
Arizona,Area,113417
Alabama,Population,3615000
Alaska,Population,365000
Arizona,Population,NA


Finally, you'll notice that these functions complain if you have one key with multiple values (e.g., if the population of Arizona were listed twice in `states`).

This is by design, but there are other packages, such as `reshape` and `reshape2`, that can handle these and apply functions to these multiple values.

They're generally more powerful, but also more confusing, functions, and we'll avoid them whenever possible.